# Numerical tessellation of waterbomb cells

In [11]:
%matplotlib widget
from bmcs_shell.api import WBElem5Param, WBElemSymb5ParamXL, WBNumericalTessellation, WBElem4Param
import numpy as np

In [2]:
data = dict(gamma=0.2, x_ur=1400, a=1000, b=1700, c=850, show_wireframe=True)
wb = WBElem5Param(**data)
# wb.interact()

In [3]:
import k3d
import random
def add_cell(plot, X_Ia, I_Fi):
    wb_mesh = k3d.mesh(X_Ia.astype(np.float32),
                             I_Fi.astype(np.uint32),
                            opacity=0.8,
                             color=0x999999,
                             side='double')
    rand_color = random.randint(0, 0xFFFFFF)
    plot += wb_mesh

    # wb_points = k3d.points(X_Ia.astype(np.float32),
    #                          color=0x999999,
    #                        point_size=100)
    # plot +=wb_points

    for I, X_a in enumerate(X_Ia):
        k3d_text = k3d.text('%g' % I, tuple(X_a), label_box=False, size=0.8, color=rand_color)
        plot += k3d_text

    wb_mesh_wireframe = k3d.mesh(X_Ia.astype(np.float32),
                                    I_Fi.astype(np.uint32),
                                    color=0x000000,
                                    wireframe=True)
    plot += wb_mesh_wireframe

In [4]:
# Source:
# https://github.com/nghiaho12/rigid_transform_3D
# http://nghiaho.com/?page_id=671
# Input: expects 3xN matrix of points
# Returns R,t
# R = 3x3 rotation matrix
# t = 3x1 column vector

def get_best_rot_and_trans_3d(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    return R, t

def get_rot_matrix_around_vector(v, angle):
    c = np.cos(angle)
    s = np.sin(angle)
    v_norm = v/np.sqrt(sum(v*v))

    # See: Rotation matrix from axis and angle (https://en.wikipedia.org/wiki/Rotation_matrix)
    cross_product_matrix = np.cross(v_norm, np.identity(v_norm.shape[0]) * -1)
    return c * np.identity(3) + s * cross_product_matrix + (1-c)*np.outer(v_norm, v_norm)

In [5]:
X_Ia = wb.X_Ia
ab0 = np.vstack((X_Ia[6], X_Ia[2])).T
ab1 = np.vstack((X_Ia[3], X_Ia[5])).T
rot, trans = get_best_rot_and_trans_3d(ab0, ab1)
ur_X_Ia = trans.flatten() + np.einsum('ba, Ia -> Ib', rot, X_Ia)

In [6]:
# Rotate around 3-5
rot_62 = get_rot_matrix_around_vector(ur_X_Ia[6] - ur_X_Ia[2], -0.3)
rot_62

array([[ 0.95735417,  0.02303576, -0.28799714],
       [-0.00450471,  0.99788535,  0.06484246],
       [ 0.28888182, -0.06077985,  0.95543346]])

In [7]:
# Bringing back to origin (because rotating is around a vector with 6-2 direction and originates from origin)
ur_X_Ia1 = ur_X_Ia - ur_X_Ia[2]
# Rotating
ur_X_Ia1 = np.einsum('ba, Ia -> Ib', rot_62, ur_X_Ia1)
# Bringing back in position
ur_X_Ia1 = ur_X_Ia1 + ur_X_Ia[2]

In [10]:
plot = k3d.plot()
I_Fi =wb.I_Fi
add_cell(plot, X_Ia, I_Fi)
add_cell(plot, ur_X_Ia1, I_Fi)
plot.display()

Output()

## Now, let's write a function to repeat the process for other cells

In [30]:
def get_cell_matching_v1_to_v2(X_Ia, v1_ids, v2_ids):
    v1_2a = np.array([X_Ia[v1_ids[0]], X_Ia[v1_ids[1]]])
    v2_2a = np.array([X_Ia[v2_ids[0]], X_Ia[v2_ids[1]]])
    rot, trans = get_best_rot_and_trans_3d(v1_2a.T, v2_2a.T)
    translated_X_Ia = trans.flatten() + np.einsum('ba, Ia -> Ib', rot, X_Ia)
    
    ####### Rotating around vector #######
    # Bringing back to origin (because rotating is around a vector originating from origin)
    translated_X_Ia1 = translated_X_Ia - translated_X_Ia[v1_ids[1]]
    
    # Rotating
    rot_around_v1 = get_rot_matrix_around_vector(translated_X_Ia1[v1_ids[0]] - translated_X_Ia1[v1_ids[1]], np.pi*1)
    translated_X_Ia1 = np.einsum('ba, Ia -> Ib', rot_around_v1, translated_X_Ia1)
    
    # Bringing back in position
    return translated_X_Ia1 + translated_X_Ia[v1_ids[1]]

In [31]:
def rotate_cell(cell_X_Ia, v1_ids, angle=np.pi):
    ####### Rotating around vector #######
    # Bringing back to origin (because rotating is around a vector originating from origin)
    cell_X_Ia_copy = np.copy(cell_X_Ia)
    cell_X_Ia = cell_X_Ia_copy - cell_X_Ia_copy[v1_ids[1]]
    
    # Rotating
    rot_around_v1 = get_rot_matrix_around_vector(cell_X_Ia[v1_ids[0]] - cell_X_Ia[v1_ids[1]], angle)
    cell_X_Ia = np.einsum('ba, Ia -> Ib', rot_around_v1, cell_X_Ia)
    
    # Bringing back in position
    return cell_X_Ia + cell_X_Ia_copy[v1_ids[1]]

In [ ]:
X_Ia = wb.X_Ia
br_X_Ia = get_cell_matching_v1_to_v2(X_Ia, np.array([4, 6]), np.array([5, 1]))
ur_X_Ia = get_cell_matching_v1_to_v2(X_Ia, np.array([6, 2]), np.array([3, 5]))
# u_X_Ia = get_cell_matching_v1_to_v2(X_Ia, np.array([1, 2]), np.array([3, 4]))

In [32]:
def rotate_and_get_diff(rotations):
    br_X_Ia_rot = rotate_cell(br_X_Ia, np.array([4, 6]), rotations[0])
    ur_X_Ia_rot = rotate_cell(ur_X_Ia, np.array([6, 2]), rotations[1])
    diff = ur_X_Ia_rot[1] - br_X_Ia_rot[3]
    dist = np.sqrt(np.sum(diff*diff))
#     print('dist=', dist)
    return dist

In [33]:
from scipy.optimize import minimize, rosen, rosen_der
x0 = [np.pi, np.pi]
res = minimize(rotate_and_get_diff, x0, tol=1e-8)
smallest_dist = res.fun
print('smallest_dist=', smallest_dist)
sol = res.x
sol

smallest_dist= 12.006910647642261


array([0.15122247, 3.03168785])

In [34]:
plot = k3d.plot()
I_Fi =wb.I_Fi
add_cell(plot, X_Ia, I_Fi)
# add_cell(plot, ur_X_Ia, I_Fi)
# add_cell(plot, br_X_Ia, I_Fi)
add_cell(plot, rotate_cell(br_X_Ia, np.array([4, 6]), sol[0]), I_Fi)
add_cell(plot, rotate_cell(ur_X_Ia, np.array([6, 2]), sol[1]), I_Fi)
plot.display()

Output()